In [2]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import read_player_stats
import training_data
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import grid_search
from sklearn import metrics
from sklearn.learning_curve import learning_curve
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 250

In [3]:
#grab total data for all positions
qb_total_df = training_data.make_total_data(seasons=range(2004,2015), pages=[0,1], pos='qb')
rb_total_df = training_data.make_total_data(seasons=range(2004,2015), pages=[0,1,2,3], pos='rb')
wr_total_df = training_data.make_total_data(seasons=range(2004,2015), pages=[0,1,2,3], pos='wr')
te_total_df = training_data.make_total_data(seasons=range(2004,2015), pages=[0,1], pos='te')

In [4]:
#training sets
qb_train_df = training_data.make_training_df(qb_total_df, seasons=range(2004,2014), ppg=True)
rb_train_df = training_data.make_training_df(rb_total_df, seasons=range(2004,2014), ppg=True)
wr_train_df = training_data.make_training_df(wr_total_df, seasons=range(2004,2014), ppg=True)
te_train_df = training_data.make_training_df(te_total_df, seasons=range(2004,2014), ppg=True)

In [5]:
qb_total_df.head()

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season
28,AJFeeley,MIA,11,191,356,1893,11,15,14,13,1,146.0,13.3,2004
164,AJFeeley,MIA,2,0,0,0,0,0,0,0,0,0.0,0.0,2005
224,AJFeeley,PHI,2,26,38,342,3,0,1,3,0,29.4,14.7,2006
304,AJFeeley,PHI,4,59,103,681,5,8,7,23,0,56.4,14.1,2007
637,AJFeeley,STL,5,53,97,548,1,2,3,4,0,31.8,6.4,2011


In [6]:
#train one regression model
X_train = np.array(qb_train_df.drop(['Name','FFPPG'], axis=1))
y_train = np.array(qb_train_df['FFPPG'])

parameters = {'alpha': np.logspace(-5,5,num=30)}
lin_model = grid_search.GridSearchCV(linear_model.Ridge(normalize=True), parameters, cv=10)
lin_model.fit(X_train, y_train)

scores =  cross_validation.cross_val_score(lin_model.best_estimator_, X_train, y_train, cv=10, scoring='mean_absolute_error')
scores

array([-5.19642518, -4.96827708, -4.85902221, -5.32455082, -4.78903028,
       -3.57224926, -4.76286332, -4.39907412, -4.39593469, -4.60734357])

In [7]:
lin_model.best_estimator_

Ridge(alpha=0.30391953823132012, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=True, solver='auto', tol=0.001)

In [8]:
qb_model = training_data.train_player_model(qb_train_df)
rb_model = training_data.train_player_model(rb_train_df)
wr_model = training_data.train_player_model(wr_train_df)
te_model = training_data.train_player_model(te_train_df)

In [9]:
#projections for 2015
qb_most_rec = training_data.data_for_projection(qb_total_df, season=2015)
rb_most_rec = training_data.data_for_projection(rb_total_df, season=2015)
wr_most_rec = training_data.data_for_projection(wr_total_df, season=2015)
te_most_rec = training_data.data_for_projection(te_total_df, season=2015)

In [10]:
qb_proj = training_data.ff_projection(qb_most_rec, qb_model)
rb_proj = training_data.ff_projection(rb_most_rec, rb_model)
wr_proj = training_data.ff_projection(wr_most_rec, wr_model)
te_proj = training_data.ff_projection(te_most_rec, te_model)

In [11]:
qb_proj

,Name,2015 Projection
50,PeytonManning,25.857974
21,DrewBrees,24.217709
2,AndrewLuck,23.341387
4,BenRoethlisberger,21.330288
0,AaronRodgers,21.320408
51,PhilipRivers,21.269777
43,MattRyan,20.611727
53,RussellWilson,20.266972
65,TonyRomo,20.255000
64,TomBrady,20.210638


In [12]:
rb_proj

,Name,2015 Projection
47,DeMarcoMurray,15.516788
99,LeVeonBell,14.998517
105,MattForte,13.147425
103,MarshawnLynch,13.002078
98,LeSeanMcCoy,12.037441
54,EddieLacy,11.952104
10,ArianFoster,11.918932
70,JamaalCharles,11.764224
78,JoiqueBell,10.114279
4,AlfredMorris,9.655172


In [13]:
wr_proj

,Name,2015 Projection
12,AntonioBrown,12.678029
47,DemaryiusThomas,12.068404
52,DezBryant,11.689193
86,JordyNelson,11.375867
5,AlshonJeffery,10.395144
141,RandallCobb,10.364489
26,CalvinJohnson,10.086471
93,JulioJones,9.910243
61,EmmanuelSanders,9.782172
0,AJGreen,9.527231


In [14]:
te_proj

,Name,2015 Projection
46,JimmyGraham,9.551878
81,RobGronkowski,9.088271
32,GregOlsen,7.803920
53,JuliusThomas,7.480508
4,AntonioGates,7.066133
21,DelanieWalker,7.052570
66,MartellusBennett,6.943020
40,JasonWitten,6.502709
15,CobyFleener,6.282668
12,CharlesClay,5.644396
